<h1 align=center>Location Recommendation Engine for opening a Restaurant</h1>

In [1]:
! pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

! pip install folium==0.11.0
import folium # plotting library

print('Libraries imported.')

Libraries imported.


In [2]:
# The code was removed by Watson Studio for sharing.

Dataset read into a pandas dataframe!


,Location,Pincode,State,District
0,A I staff colony,400029,Maharashtra,Mumbai
1,Aareymilk Colony,400065,Maharashtra,Mumbai
2,Agripada,400011,Maharashtra,Mumbai
3,Airport,400099,Maharashtra,Mumbai
4,Ambewadi,400004,Maharashtra,Mumbai


In [3]:
# The code was removed by Watson Studio for sharing.

In [4]:
#find the cordinates of all the location
#Set the cordinates as 0 for the location that is not found by geopy
myAddress = df['Location'] + ', ' + df['District']

loc_latitude = []
loc_longitude = []
geolocator = Nominatim(user_agent="Mumbai_explorer")
for myA in myAddress:
    try:
        location = geolocator.geocode(myA)
        latitude = location.latitude
        longitude = location.longitude
        loc_latitude.append(latitude)
        loc_longitude.append(longitude)

    except Exception as e:
        latitude = 0
        longitude = 0
        loc_latitude.append(latitude)
        loc_longitude.append(longitude)

df['Latitude']= loc_latitude
df['Longitude']= loc_longitude
print ('Records in the dataframe are', df.shape)
df.head()

Records in the dataframe are (182, 6)


,Location,Pincode,State,District,Latitude,Longitude
0,A I staff colony,400029,Maharashtra,Mumbai,0.000000,0.000000
1,Aareymilk Colony,400065,Maharashtra,Mumbai,0.000000,0.000000
2,Agripada,400011,Maharashtra,Mumbai,18.975302,72.824898
3,Airport,400099,Maharashtra,Mumbai,19.090131,72.863703
4,Ambewadi,400004,Maharashtra,Mumbai,19.186776,72.859313


In [5]:
df_final = df[df['Latitude'] != 0]
df_final = df_final.reset_index(drop=True)
print ('Records after dropping the rows with cordinates as 0 are', df_final.shape)
df_final.head()

Records after dropping the rows with cordinates as 0 are (138, 6)


,Location,Pincode,State,District,Latitude,Longitude
0,Agripada,400011,Maharashtra,Mumbai,18.975302,72.824898
1,Airport,400099,Maharashtra,Mumbai,19.090131,72.863703
2,Ambewadi,400004,Maharashtra,Mumbai,19.186776,72.859313
3,Andheri,400053,Maharashtra,Mumbai,19.119698,72.846420
4,Andheri East,400069,Maharashtra,Mumbai,19.115883,72.854202


In [6]:
latitude = df_final.loc[0, 'Latitude']
longitude = df_final.loc[0, 'Longitude']
map_mumbai = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df['Latitude'],df['Longitude'],df['Location']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_mumbai)  
    
map_mumbai

In [7]:
LIMIT = 500 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
results = requests.get(url).json()

In [8]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
from pandas.io.json import json_normalize

In [9]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))
nearby_venues.head()

33 venues were returned by Foursquare.


,name,categories,lat,lng
0,Celejor,Bakery,18.975844,72.823679
1,Tote On The Turf,Nightclub,18.980266,72.820294
2,cafe coffee day,Coffee Shop,18.976988,72.824051
3,Mahalaxmi Race Course (Royal Western India Tur...,Club House,18.980535,72.818588
4,Neel,Indian Restaurant,18.980407,72.820403


In [10]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):           
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [12]:
Mumbai_venues = getNearbyVenues(names=df_final['Location'], latitudes=df_final['Latitude'], longitudes=df_final['Longitude'])
neighborhood_latitude = df_final.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_final.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_final.loc[0, 'Location'] # neighborhood name

print('the total records found are ',Mumbai_venues.shape[0])

the total records found are  2052


In [13]:
Mumbai_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agripada,4,4,4,4,4,4
Airport,4,4,4,4,4,4
Ambewadi,3,3,3,3,3,3
Andheri,10,10,10,10,10,10
Andheri East,13,13,13,13,13,13
Andheri Railway station,10,10,10,10,10,10
Antop Hill,6,6,6,6,6,6
Asvini,3,3,3,3,3,3
Azad Nagar,21,21,21,21,21,21


In [14]:
print('There are {} uniques categories.'.format(len(Mumbai_venues['Venue Category'].unique())))

There are 211 uniques categories.


In [15]:
Mumbai_onehot = pd.get_dummies(Mumbai_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Mumbai_onehot['Neighborhood'] = Mumbai_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Mumbai_onehot.columns[-1]] + list(Mumbai_onehot.columns[:-1])
Mumbai_onehot = Mumbai_onehot[fixed_columns]
Mumbai_onehot.head()

,Yoga Studio,ATM,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Arcade,Art Gallery,...,Trail,Train,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
Mumbai_grouped = Mumbai_onehot.groupby('Neighborhood').mean().reset_index()
Mumbai_grouped

,Neighborhood,Yoga Studio,ATM,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Arcade,...,Trail,Train,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store
0,Agripada,0.000000,0.0,0.00,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.00000,0.000000,0.000000,0.000000
1,Airport,0.000000,0.0,0.75,0.00000,0.25,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.00000,0.000000,0.000000,0.000000
2,Ambewadi,0.000000,0.0,0.00,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.00000,0.000000,0.000000,0.333333
3,Andheri,0.000000,0.0,0.00,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.00000,0.000000,0.000000,0.000000
4,Andheri East,0.000000,0.0,0.00,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.00000,0.000000,0.000000,0.000000
5,Andheri Railway station,0.000000,0.0,0.00,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.00000,0.000000,0.000000,0.000000
6,Antop Hill,0.000000,0.0,0.00,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.166667,0.0,0.000000,0.000000,0.000000,0.00,0.00000,0.000000,0.000000,0.000000
7,Asvini,0.000000,0.0,0.00,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.00000,0.000000,0.000000,0.000000
8,Azad Nagar,0.000000,0.0,0.00,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.047619,0.000000,0.00,0.00000,0.000000,0.000000,0.000000
9,B P t colony,0.000000,0.0,0.00,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.00000,0.000000,0.000000,0.000000


In [17]:
num_top_venues = 5

for hood in Mumbai_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Mumbai_grouped[Mumbai_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agripada----
                venue  freq
0        Soccer Field  0.25
1         Coffee Shop  0.25
2  Athletics & Sports  0.25
3              Bakery  0.25
4         Yoga Studio  0.00


----Airport----
             venue  freq
0          Airport  0.75
1  Airport Service  0.25
2      Yoga Studio  0.00
3      Opera House  0.00
4    Movie Theater  0.00


----Ambewadi----
               venue  freq
0      Women's Store  0.33
1   Department Store  0.33
2  Indian Restaurant  0.33
3           Wine Bar  0.00
4             Office  0.00


----Andheri----
                  venue  freq
0     Indian Restaurant   0.2
1  Fast Food Restaurant   0.2
2     Indian Sweet Shop   0.1
3                Bakery   0.1
4            Food Court   0.1


----Andheri East----
                venue  freq
0   Indian Restaurant  0.23
1  Light Rail Station  0.08
2   Electronics Store  0.08
3  Chinese Restaurant  0.08
4        Camera Store  0.08


----Andheri Railway station----
                  venue  freq
0     Indian 

In [18]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [19]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Mumbai_grouped['Neighborhood']

for ind in np.arange(Mumbai_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Mumbai_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agripada,Coffee Shop,Bakery,Athletics & Sports,Soccer Field,Women's Store,Farmers Market,Food Stand,Food Court,Food & Drink Shop,Food
1,Airport,Airport,Airport Service,Women's Store,Farm,Food Stand,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market
2,Ambewadi,Women's Store,Indian Restaurant,Department Store,Falafel Restaurant,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish & Chips Shop
3,Andheri,Fast Food Restaurant,Indian Restaurant,Sandwich Place,Bakery,Food Court,Juice Bar,Restaurant,Indian Sweet Shop,Fish & Chips Shop,Field
4,Andheri East,Indian Restaurant,Pizza Place,Gym / Fitness Center,Camera Store,Light Rail Station,Shopping Mall,Chinese Restaurant,Electronics Store,BBQ Joint,Hotel
5,Andheri Railway station,Fast Food Restaurant,Indian Restaurant,Sandwich Place,Bakery,Food Court,Juice Bar,Restaurant,Indian Sweet Shop,Fish & Chips Shop,Field
6,Antop Hill,Indian Restaurant,Multiplex,Trail,Bus Station,Gym / Fitness Center,Grocery Store,Women's Store,Falafel Restaurant,Food,Flower Shop
7,Asvini,Soccer Field,Fast Food Restaurant,Bus Station,Food Stand,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish & Chips Shop
8,Azad Nagar,Bar,Asian Restaurant,Coffee Shop,Pizza Place,Bowling Alley,Sandwich Place,Snack Place,Burger Joint,Falafel Restaurant,Chinese Restaurant
9,B P t colony,Hotel,Convenience Store,Food Stand,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish & Chips Shop,Field


In [20]:
from sklearn.cluster import KMeans

kclusters = 5

Mumbai_grouped_clustering = Mumbai_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Mumbai_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 1, 3, 3, 3, 3, 3, 2, 2, 4], dtype=int32)

In [21]:
Mumbai_merged = pd.merge(left=neighborhoods_venues_sorted, right=df_final, left_on='Neighborhood', right_on='Location')

# add clustering labels
Mumbai_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood

Mumbai_merged.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Location,Pincode,State,District,Latitude,Longitude,Cluster Labels
0,Agripada,Coffee Shop,Bakery,Athletics & Sports,Soccer Field,Women's Store,Farmers Market,Food Stand,Food Court,Food & Drink Shop,Food,Agripada,400011,Maharashtra,Mumbai,18.975302,72.824898,2
1,Airport,Airport,Airport Service,Women's Store,Farm,Food Stand,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Airport,400099,Maharashtra,Mumbai,19.090131,72.863703,1
2,Ambewadi,Women's Store,Indian Restaurant,Department Store,Falafel Restaurant,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish & Chips Shop,Ambewadi,400004,Maharashtra,Mumbai,19.186776,72.859313,3
3,Andheri,Fast Food Restaurant,Indian Restaurant,Sandwich Place,Bakery,Food Court,Juice Bar,Restaurant,Indian Sweet Shop,Fish & Chips Shop,Field,Andheri,400053,Maharashtra,Mumbai,19.119698,72.846420,3
4,Andheri East,Indian Restaurant,Pizza Place,Gym / Fitness Center,Camera Store,Light Rail Station,Shopping Mall,Chinese Restaurant,Electronics Store,BBQ Joint,Hotel,Andheri East,400069,Maharashtra,Mumbai,19.115883,72.854202,3


In [22]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
import matplotlib.cm as cm
from matplotlib import colors

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Mumbai_merged['Latitude'], Mumbai_merged['Longitude'], Mumbai_merged['Neighborhood'], Mumbai_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [23]:
Mumbai_merged.loc[Mumbai_merged['Cluster Labels'] == 0, Mumbai_merged.columns[[14]+[0]+[13]+list(range(1, 11))]]

,Latitude,Neighborhood,State,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,19.226723,Borivali East,Maharashtra,Indian Restaurant,Gym,Women's Store,Falafel Restaurant,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish & Chips Shop
25,19.214119,Charkop,Maharashtra,Indian Restaurant,Pharmacy,Café,Bakery,Miscellaneous Shop,Farm,Food Court,Food & Drink Shop,Food,Flower Shop
51,18.972063,Haines Road,Maharashtra,Indian Restaurant,Pizza Place,Cupcake Shop,Coffee Shop,Asian Restaurant,Bar,Farm,Food Court,Food & Drink Shop,Food
52,18.948282,Hanuman Road,Maharashtra,Indian Restaurant,Bar,Snack Place,Wine Shop,Cheese Shop,Ice Cream Shop,American Restaurant,Food,Restaurant,Market
63,18.949258,Kalbadevi,Maharashtra,Indian Restaurant,Food,Market,Café,Cheese Shop,Women's Store,Farm,Food Court,Food & Drink Shop,Flower Shop
69,19.061218,Kherwadi,Maharashtra,Indian Restaurant,Café,Pizza Place,Bar,Farm,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market
75,18.954146,Madhavbaug,Maharashtra,Indian Restaurant,Grocery Store,Food,Arcade,Fast Food Restaurant,Event Space,Food Court,Food & Drink Shop,Flower Shop,Flea Market
82,18.955056,Mandvi,Maharashtra,Indian Restaurant,Smoke Shop,Café,Juice Bar,Arcade,Sandwich Place,Chinese Restaurant,BBQ Joint,Dessert Shop,Convenience Store
95,19.122837,Nagardas Road,Maharashtra,Indian Restaurant,Sandwich Place,Juice Bar,Women's Store,Falafel Restaurant,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish & Chips Shop


In [24]:
Mumbai_merged.loc[Mumbai_merged['Cluster Labels'] == 1, Mumbai_merged.columns[[14]+[0]+[13]+list(range(1, 11))]]

,Latitude,Neighborhood,State,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,19.090131,Airport,Maharashtra,Airport,Airport Service,Women's Store,Farm,Food Stand,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market
55,19.090131,International Airport,Maharashtra,Airport,Airport Service,Women's Store,Farm,Food Stand,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market


In [25]:
Mumbai_merged.loc[Mumbai_merged['Cluster Labels'] == 2, Mumbai_merged.columns[[14]+[0]+[13]+list(range(1, 11))]]

,Latitude,Neighborhood,State,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,18.975302,Agripada,Maharashtra,Coffee Shop,Bakery,Athletics & Sports,Soccer Field,Women's Store,Farmers Market,Food Stand,Food Court,Food & Drink Shop,Food
7,18.900689,Asvini,Maharashtra,Soccer Field,Fast Food Restaurant,Bus Station,Food Stand,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish & Chips Shop
8,19.128315,Azad Nagar,Maharashtra,Bar,Asian Restaurant,Coffee Shop,Pizza Place,Bowling Alley,Sandwich Place,Snack Place,Burger Joint,Falafel Restaurant,Chinese Restaurant
10,5.416873,B.P.lane,Maharashtra,Café,Public Art,Hostel,Indian Restaurant,Hotel,Bed & Breakfast,Food Truck,Bar,Vegetarian / Vegan Restaurant,Halal Restaurant
13,19.168814,Bangur Nagar,Maharashtra,Coffee Shop,Department Store,Clothing Store,Women's Store,Farm,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market
15,19.152814,Best Staff colony,Maharashtra,Clothing Store,Café,Asian Restaurant,Bakery,Fast Food Restaurant,Women's Store,Farmers Market,Food Stand,Food Court,Food & Drink Shop
16,19.063330,Bharat Nagar,Maharashtra,Pizza Place,Fast Food Restaurant,Café,Food Stand,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish & Chips Shop,Field
19,19.229068,Borivali,Maharashtra,Chinese Restaurant,Ice Cream Shop,Clothing Store,Food Truck,Café,Burger Joint,Fast Food Restaurant,Gym,Restaurant,Vegetarian / Vegan Restaurant
22,19.008301,Century Mill,Maharashtra,Lounge,Café,Bar,Restaurant,Chinese Restaurant,Athletics & Sports,Thai Restaurant,Clothing Store,Nightclub,Bakery
23,19.115287,Chakala Midc,Maharashtra,Hotel,Pizza Place,Indian Restaurant,Multiplex,Café,Fast Food Restaurant,Restaurant,Seafood Restaurant,Ice Cream Shop,Asian Restaurant


In [26]:
Mumbai_merged.loc[Mumbai_merged['Cluster Labels'] == 3, Mumbai_merged.columns[[14]+[0]+[13]+list(range(1, 11))]]

,Latitude,Neighborhood,State,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,19.186776,Ambewadi,Maharashtra,Women's Store,Indian Restaurant,Department Store,Falafel Restaurant,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish & Chips Shop
3,19.119698,Andheri,Maharashtra,Fast Food Restaurant,Indian Restaurant,Sandwich Place,Bakery,Food Court,Juice Bar,Restaurant,Indian Sweet Shop,Fish & Chips Shop,Field
4,19.115883,Andheri East,Maharashtra,Indian Restaurant,Pizza Place,Gym / Fitness Center,Camera Store,Light Rail Station,Shopping Mall,Chinese Restaurant,Electronics Store,BBQ Joint,Hotel
5,19.119698,Andheri Railway station,Maharashtra,Fast Food Restaurant,Indian Restaurant,Sandwich Place,Bakery,Food Court,Juice Bar,Restaurant,Indian Sweet Shop,Fish & Chips Shop,Field
6,19.020761,Antop Hill,Maharashtra,Indian Restaurant,Multiplex,Trail,Bus Station,Gym / Fitness Center,Grocery Store,Women's Store,Falafel Restaurant,Food,Flower Shop
11,19.058336,Bandra West,Maharashtra,Indian Restaurant,Café,Chinese Restaurant,Bakery,Clothing Store,Bar,Pub,Asian Restaurant,Fried Chicken Joint,Bookstore
12,19.061657,Bandra(east),Maharashtra,Indian Restaurant,Pizza Place,Indie Movie Theater,Café,Diner,Italian Restaurant,General Entertainment,Bistro,Ice Cream Shop,Farm
14,18.934319,Bazargate,Maharashtra,Indian Restaurant,Café,Seafood Restaurant,Parsi Restaurant,Irani Cafe,BBQ Joint,Lounge,Clothing Store,Boutique,Coffee Shop
17,19.015786,Bhawani Shankar,Maharashtra,Shopping Mall,Indian Restaurant,Clothing Store,Asian Restaurant,Women's Store,Farm,Food Court,Food & Drink Shop,Food,Flower Shop
18,19.019009,Bhawani Shankar rd,Maharashtra,Indian Restaurant,Fast Food Restaurant,Movie Theater,Bakery,Women's Store,Electronics Store,Flower Shop,Maharashtrian Restaurant,Restaurant,Bar


In [27]:
Mumbai_merged.loc[Mumbai_merged['Cluster Labels'] == 4, Mumbai_merged.columns[[14]+[0]+[13]+list(range(1, 11))]]

,Latitude,Neighborhood,State,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,19.101937,B P t colony,Maharashtra,Hotel,Convenience Store,Food Stand,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish & Chips Shop,Field


# Conclusion

Based on the Clusters formed it would be a good idea to open a chinese restaurant in Clusters 2 & 3 since the other clusters already have Chinese Restaurants or high number of Indian restaurant that may serve Chinese also in their vicinities.

## Thank you

This notebook was created by <b>Mohammed Tausif Khan</b>